# From Notebook to Kubeflow Pipeline using Fashion MNIST

In this notebook, we will walk you through the steps of converting a machine learning model, which you may already have on a jupyter notebook, into a Kubeflow pipeline. As an example, we will make use of the fashion we will make use of the fashion MNIST dataset and the [Basic classification with Tensorflow](https://www.tensorflow.org/tutorials/keras/classification) example.

In this example we use:

* **Kubeflow pipelines** - [Kubeflow Pipelines](https://www.kubeflow.org/docs/pipelines/overview/pipelines-overview/) is a machine learning workflow platform that is helping data scientists and ML engineers tackle experimentation and productionization of ML workloads. It allows users to easily orchestrate scalable workloads using an SDK right from the comfort of a Jupyter Notebook.

* **Microk8s** - [Microk8s](https://microk8s.io/docs) is a service that gives you the ability to spin up a lightweight Kubernetes cluster right on your local machine. It comes with Kubeflow built right in. 

**Note:** This notebook is to be run on a notebook server inside the Kubeflow environment. 

## Section 1: Data exploration (as in [here](https://www.tensorflow.org/tutorials/keras/classification))

The [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist)  dataset contains 70,000 grayscale images in 10 clothing categories. Each image is 28x28 pixels in size. We chose this dataset to demonstrate the funtionality of Kubeflow Pipelines without introducing too much complexity in the implementation of the ML model.

To familiarize you with the dataset we will do a short exploration. It is always a good idea to understand your data before you begin any kind of analysis.

<table>
  <tr><td>
    <img src="https://tensorflow.org/images/fashion-mnist-sprite.png"
         alt="Fashion MNIST sprite"  width="600">
  </td></tr>
  <tr><td align="center">
    <b>Figure 1.</b> <a href="https://github.com/zalandoresearch/fashion-mnist">Fashion-MNIST samples</a> (by Zalando, MIT License).<br/>&nbsp;
  </td></tr>
</table>


### 1.1 Install packages:

In [ ]:
!python -m pip install --user --upgrade pip
!pip install --user --upgrade pandas matplotlib numpy

After the installation, we need to restart kernel for changes to take effect:

In [ ]:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

### 1.2 Import libraries

In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Data exploration libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### 1.2 Import the Fashion MNIST dataset

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

Each image is mapped to a single label. Since the *class names* are not included with the dataset, store them here to use later when plotting the images:

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

Let's look at the format of each dataset split. The training set contains 60,000 images and the test set contains 10,000 images which are each 28x28 pixels.

---



### 1.3 Explore the data

In [ ]:
print(f'Number of training images: {train_images.shape[0]}\n')
print(f'Number of test images: {test_images.shape[0]}\n')

print(f'Image size: {train_images.shape[1:]}')


There are logically 60,000 training labels and 10,000 test labels.

In [ ]:
print(f'Number of labels: {len(train_labels)}\n')
print(f'Number of test labels: {len(test_labels)}')

Each label is an integer between 0 and 9 corresponding to the 10 class names.

In [ ]:
unique_train_labels = np.unique(train_labels)

for label in zip(class_names, train_labels):
  label_name, label_num = label
  print(f'{label_name}: {label_num}')

### 1.4 Preprocess the data

To properly train the model, the data must be normalized so each value will fall between 0 and 1. Later on this step will be done inside of the training script but we will show what that process looks like here.

The first image shows that the values fall in a range between 0 and 255.

In [ ]:
plt.figure()
plt.imshow(train_images[0])
plt.colorbar()
plt.grid(False)
plt.show()

To scale the data we divide the training and test values by 255.

In [ ]:
train_images = train_images / 255.0

test_images = test_images / 255.0

We plot the first 25 images from the training set to show that the data is in fact in the form we expect.

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

# Section 2: Kubeflow pipeline building

Up until this point, all our steps are similar to what you can find in the [Basic classification with Tensorflow](https://https://www.tensorflow.org/tutorials/keras/classification) example. The next step on that example is to build the model, and that is what we will do. However, we will make use of the containerized approach provided by Kubeflow to allow our model to be run using Kubernetes.

### 2.1 Install Kubeflow pipelines SDK

Now that we have a good handle on what the data looks like. The first step is to install the Kubeflow Pipelines SDK package.



In [4]:
!pip install --user --upgrade kfp

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Requirement already up-to-date: kfp in ./.local/lib/python3.6/site-packages (0.5.1)


After the installation, we need to restart kernel for changes to take effect:

In [ ]:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

If the install was successful then you should see:

**/usr/local/bin/dsl-compile**

In [1]:
!which dsl-compile

/usr/local/bin/dsl-compile


### 2.2 Import kfp

In [2]:
# Kubeflow SDK
import kfp
import kfp.dsl as dsl


# Helper libraries
import os
import subprocess

### 2.3 Build Container Images using Google Cloud Build

Cloud Build is a wonderful service offerred by Google Cloud Platform. It allows you to build containers in the cloud which allows you to build large containers without local compute constraints. Your images are then stored in a secure private Google Container Registry. The only charges that apply are those pertaining to storing the images on Google Cloud Storage.

In [3]:
PROJECT_ID = 'manceps-labs'

Open a new terminal and enter `gcloud init` to login to your Google Cloud Account before running the next cell.

In [ ]:
# Enable the Cloud Build and Container Registry APIs
!gcloud services enable cloudbuild.googleapis.com containerregistry.googleapis.com

# Get Cloud Build service account email.
SERVICE_ACCOUNT_EMAIL=!gcloud iam service-accounts list | grep 'cloud-build' | awk '{ print $2 }'

Since the only services we are using are Cloud Build and Container Registry, set the default credentials to use the default Cloud Build service account. This will narrow the scope of what the application is capable of doing on Google Cloud Platform.

In [ ]:
!gcloud iam service-accounts keys create key.json \
    --iam-account={SERVICE_ACCOUNT_EMAIL[0]}

!export GOOGLE_APPLICATION_CREDENTIALS=key.json

Define your local paths where your component files will be stored. In our case we will be creating two components. The first component will be a training component which will pull the data, process it, and train a model. 

The second component will be a prediction component which will acually be predicting the label of a specified image from the test data.

In [4]:
TRAINING_PATH = './tmp/components/mnist_training/'
PREDICT_PATH = './tmp/components/mnist_predict/'
DATA_PATH = '/mnt'

### 2.4 


The following cells define strings that will be written to the training and test *app.py* file. It is recommended to look at the corresponding Fashion MNIST notebook to match the code to understand what was included from there and what needed to be added to ensure that it runs properly during pipeline execution.

In [5]:
train_file = '''import argparse
import pickle

import tensorflow as tf
from tensorflow import keras

# Use the argparse package to define command line arguments.
parser = argparse.ArgumentParser()

parser.add_argument('--epochs', type=str, required=False, default=10, help='Number of epochs to train the model for.')
parser.add_argument('--data-path', type=str, required=True, help='Absolute path where the persistent volume will be mounted.')
parser.add_argument('--model-file', type=str, required=True, help='Name of the model file (ex. model.h5).')
    
args = parser.parse_args()

epochs = args.epochs
data_path = args.data_path
model_file = args.model_file

# Download the dataset and split into training and test data. 
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Normalize the data so that the values all fall between 0 and 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the model using Keras.
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(512, activation=tf.nn.relu),
    keras.layers.Dropout(0.25),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

print(model.summary())    

# Run a training job with specified number of epochs (Default = 10)
model.fit(train_images, train_labels, epochs=epochs)

# Evaluate the model and print the results
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print('Test accuracy:', test_acc)

# Save the model to the designated 
model.save(f'{data_path}/{model_file}')

# Save the test_data as a pickle file to be used by the predict component.
with open(f'{data_path}/test_data', 'wb') as f:
    pickle.dump((test_images,test_labels), f)

'''

In [18]:
predict_file = '''import argparse
import pickle

import tensorflow as tf
from tensorflow import keras

import numpy as np

# Use the argparse package to define command line arguments.
parser = argparse.ArgumentParser()

parser.add_argument('--data-path', type=str, required=True, help='Absolute path where the persistent volume will be mounted.')
parser.add_argument('--image-number', type=int, required=False, default=0, help='Image to predict (0-9999).')
parser.add_argument('--model-file', type=str, required=True, help='Name of the saved Keras model file (ex. model.h5).')

args = parser.parse_args()

data_path = args.data_path
image_number = args.image_number
model_file = args.model_file

# Load the saved Keras model
model = keras.models.load_model(f'{data_path}/{model_file}')

# Load and unpack the test_data
with open(f'{data_path}/test_data','rb') as f:
    test_data = pickle.load(f)

# Separate the test_images from the test_labels.
test_images, test_labels = test_data

# Define the class names.
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# Define a Softmax layer to define outputs as probabilities
probability_model = tf.keras.Sequential([model, 
                                        tf.keras.layers.Softmax()])

# Grab an image from the test dataset.
img = test_images[image_number]

# Add the image to a batch where it is the only member.
img = (np.expand_dims(img,0))

# Predict the label of the image.
predictions = probability_model.predict(img)

# Take the prediction with the highest probability
prediction = np.argmax(predictions[0])

# Retrieve the true label of the image from the test labels.
true_label = test_labels[image_number]

with open(f'{data_path}/result.txt', 'w') as f:
  f.write("Prediction: {} | Confidence: {:2.0f}% | Actual: {}".format(class_names[prediction],
                                100*np.max(predictions),
                                class_names[true_label]))
'''

### 2.5 Create docker files

In [19]:
dockerfile ='''FROM tensorflow/tensorflow:latest-gpu-py3

# Change current working directory.
WORKDIR /opt

COPY . /opt
'''



In [20]:
def create_files(path, app_text, dockerfile=dockerfile):
    '''
    Creates app.py file and Dockerfile at specified path.

    Args:
      path (str): Destination path where app.py and Dockerfile will be created.
      app_text (str): A string to be used as the bodyReturns:
        str: User choice.of the app.py file.
      dockerfile (str): A string specifying the Dockerfile structure to be used.

    Returns:
      None
    '''
    
    # Install Numpy if building predict container
    if app_text.find('numpy') != -1:
        dockerfile += 'RUN python3 -m pip install numpy matplotlib'
    
    # Create parent directories
    if not os.path.isdir(path):
        os.makedirs(path)
    
    # Create the app.py code file from app_text
    with open(f'{path}/app.py', 'w') as f:
        f.writelines(app_text)
    
    # Create Dockerfile from dockerfile
    with open(f'{path}/Dockerfile', 'w') as f:
        f.writelines(dockerfile)
    
    return print('File path: ', path, '\n' \
                 'Files created: ', os.listdir(path), '\n')

Create the files needed to build our component containers.

In [21]:
create_files(TRAINING_PATH, train_file)
create_files(PREDICT_PATH, predict_file)

File path:  ./tmp/components/mnist_training/ 
Files created:  ['app.py', 'Dockerfile'] 

File path:  ./tmp/components/mnist_predict/ 
Files created:  ['app.py', 'Dockerfile'] 



In [22]:
def create_image_string(image_name, project_id):
    '''
    Creates Docker image string.
    
    Args:
      image_name (str): Name for Docker image.
      project_id (str): GCP Project ID.

    Returns:
      str: A GCR Docker image tag.
    '''
    gcr_image = f"us.gcr.io/{project_id}/{image_name}:latest"
    print(gcr_image)
    return gcr_image


def build_docker_image(gcr_image, app_folder):
    '''
    Builds Docker image using Google Cloud Build and stores in Google Cloud Repository

    Args:
      gcr_image (str): A Google Container Registry tag for Docker container build.
      app_folder (str): Path to app.py and Dockerfile.

    Return:
      None
    '''
    cmd = ['gcloud', 'builds', 'submit', '--tag', gcr_image, app_folder]

    build_log = subprocess.run(cmd, stdout=subprocess.PIPE).stdout[:-1].decode('utf-8')
    print(build_log)

Here we produce the Docker image tags we will use to store our images on Google Container Registry.

In [23]:
train_gcr_image = create_image_string('train', PROJECT_ID)
predict_gcr_image = create_image_string('predict', PROJECT_ID)

us.gcr.io/manceps-labs/train:latest
us.gcr.io/manceps-labs/predict:latest


Now we build the train and predict containers we will be using in our Kubeflow Pipeline. Make sure that the message that you receive once the operation is complete signals that the builds were successful before moving on. You can check out the [Google Cloud Platform console](https://console.cloud.google.com) to see the logs streaming live.

In [24]:
#build_docker_image(train_gcr_image, TRAINING_PATH)
build_docker_image(predict_gcr_image, PREDICT_PATH)

----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "31d3c371-b74c-44ec-a07d-2a5c3a991624"

FETCHSOURCE
Fetching storage object: gs://manceps-labs_cloudbuild/source/1593006561.79-f5944146e5ae44ec9bdebca49c86b667.tgz#1593006562169666
Copying gs://manceps-labs_cloudbuild/source/1593006561.79-f5944146e5ae44ec9bdebca49c86b667.tgz#1593006562169666...
/ [1 files][  1.1 KiB/  1.1 KiB]                                                
Operation completed over 1 objects/1.1 KiB.                                      
BUILD
Already have image (with digest): gcr.io/cloud-builders/docker

                   ***** NOTICE *****

Alternative official `docker` images, including multiple versions across
multiple platforms, are maintained by the Docker Team. For details, please
visit https://hub.docker.com/_/docker.

                ***** END OF NOTICE *****


Step 1/4 : FROM tensorflow/tensorflow:latest-gpu-py3
latest-gpu-py3: Pulling from tensorflow/tensorflow


### 2.6 Define Kubeflow Pipeline

Create a client to communicate with the Pipelines API server.

In [25]:
client = kfp.Client(host='pipelines-api.kubeflow.svc.cluster.local:8888')

Kubeflow Pipelines are created decalaratively. The code is not run until the pipeline is compiled. 

Define some environment variables which are to be used as inputs at various points in the pipeline.

In [26]:
# Name of the Keras model that will be saved.
MODEL_FILE = 'mnist_model.h5'

# An integer representing an image from the test set that the model will attempt to predict the label for.
IMAGE_NUMBER = 1

Our next step will be to create the various components that will make up the pipeline. Define the pipeline using the *@dsl.pipeline* decorator.

The pipeline function is defined and includes a number of paramters that will be fed into our various components throughout execution.

A [Persistent Volume Claim](https://kubernetes.io/docs/concepts/storage/persistent-volumes/) is created using the [VolumeOp](https://) method to save our data across our components. Note that while this is a great method to use locally, you could also use a cloud bucket for your persistent storage.

In [34]:
# Define the pipeline
@dsl.pipeline(
   name='MNIST Pipeline',
   description='A toy pipeline that performs mnist model training and prediction.'
)

# Define parameters to be fed into pipeline
def mnist_container_pipeline(
    data_path: str = DATA_PATH,
    model_file: str = MODEL_FILE, 
    project_id: str = PROJECT_ID,
    image_number: int = IMAGE_NUMBER
):
    
    # Define volume to share data between components.
    vop = dsl.VolumeOp(
    name="create_volume",
    resource_name="data-volume", 
    size="1Gi", 
    modes=dsl.VOLUME_MODE_RWM)
    
    # Create MNIST training component.
    mnist_training_container = dsl.ContainerOp(
      name="model_training",
      image='us.gcr.io/{}/train:latest'.format(project_id),
      command=['python3', '/opt/app.py'],
      pvolumes={data_path: vop.volume}, # Mount volume to specific data path.
      arguments=['--data-path', data_path,
                 '--model-file', model_file],
    )

    # Create MNIST prediction component.
    mnist_predict_container = dsl.ContainerOp(
        name="prediction",
        image='us.gcr.io/{}/predict:latest'.format(project_id),
        command=['python3', '/opt/app.py'],
        pvolumes={data_path: mnist_training_container.pvolume}, # Mount same volume from training component.
        arguments=['--data-path', data_path,
                   '--image-number', image_number,
                   '--model-file', model_file]
    )
    
    # Print the result of the prediction
    mnist_result_container = dsl.ContainerOp(
        name="print_prediction",
        image='library/bash:4.4.23',
        pvolumes={data_path: mnist_predict_container.pvolume}, # Mount the same volume from the training and predict components.
        arguments=['cat', f'{data_path}/result.txt']
    )

### 2.7 Run pipeline

Finally we feed our pipeline definition into the compiler and run it as an experiment. This will give us 2 links at the bottom that we can follow to the [Kubeflow Pipelines UI](https://www.kubeflow.org/docs/pipelines/overview/pipelines-overview/) where you can check logs, artifacts, inputs/outputs, and visually see the progress of your pipeline.

In [35]:
pipeline_func = mnist_container_pipeline

In [36]:
experiment_name = 'minist_kubeflow'
run_name = pipeline_func.__name__ + ' run'

arguments = {"model_file":MODEL_FILE,
             "project_id":PROJECT_ID,
             "image_number": 1}

# Submit pipeline directly from pipeline function
run_result = client.create_run_from_pipeline_func(pipeline_func, 
                                                  experiment_name=experiment_name, 
                                                  run_name=run_name, 
                                                  arguments=arguments)